In [10]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [11]:
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='movies')
cur = con.cursor()

# Q1: how can i outer join the revenue tables to have a full picture of revenue

In [12]:
query2 = """SELECT 
    CASE
        WHEN title IS NULL THEN movie
        ELSE title
    END AS movie_title,
    release_date,
    year,
    production_budget,
    CASE
        WHEN title IS NULL THEN tmdb_domestic_gross
        ELSE bom_domestic_gross
    END AS domestic_gross,
    CASE
        WHEN title IS NULL THEN tmdb_worldwide_gross
        ELSE bom_domestic_gross + bom_foreign_gross
    END AS worldwide_gross
FROM
    (SELECT DISTINCT
        title,
            movie,
            CASE
                WHEN b.domestic_gross IS NULL THEN 0
                ELSE b.domestic_gross
            END AS bom_domestic_gross,
            CASE
                WHEN b.foreign_gross IS NULL THEN 0
                ELSE b.foreign_gross
            END AS bom_foreign_gross,
            release_date,
            year,
            production_budget,
            t.domestic_gross AS tmdb_domestic_gross,
            t.worldwide_gross AS tmdb_worldwide_gross
    FROM
        bom_movie_gross b
    LEFT JOIN tn_movie_budgets t ON movie = title
    WHERE
        b.domestic_gross IS NOT NULL
            OR b.foreign_gross IS NOT NULL
            OR t.domestic_gross IS NOT NULL
            OR t.worldwide_gross IS NOT NULL UNION SELECT DISTINCT
        title,
            movie,
            CASE
                WHEN b.domestic_gross IS NULL THEN 0
                ELSE b.domestic_gross
            END AS bom_domestic_gross,
            CASE
                WHEN b.foreign_gross IS NULL THEN 0
                ELSE b.foreign_gross
            END AS bom_foreign_gross,
            release_date,
            year,
            production_budget,
            t.domestic_gross AS tmdb_domestic_gross,
            t.worldwide_gross AS tmdb_worldwide_gross
    FROM
        bom_movie_gross b
    RIGHT JOIN tn_movie_budgets t ON movie = title
    WHERE
        b.domestic_gross IS NOT NULL
            OR b.foreign_gross IS NOT NULL
            OR t.domestic_gross IS NOT NULL
            OR t.worldwide_gross IS NOT NULL
    ORDER BY bom_domestic_gross) a
"""
df_joint_rev =  pd.read_sql_query(query2, con)

In [13]:
df_joint_rev[df_joint_rev.columns[3:]] = df_joint_rev[df_joint_rev.columns[3:]].replace('[\$,]', '', regex=True).astype(float)
#turns rev from string to integer
df_joint_rev.head()

,movie_title,release_date,year,production_budget,domestic_gross,worldwide_gross
0,How She Move,"Jan 25, 2008",NaN,17000000.0,7070641.0,8607815.0
1,Mata Hari,"Dec 26, 1931",NaN,558000.0,900000.0,900000.0
2,Stargate,"Oct 28, 1994",NaN,55000000.0,71565669.0,196565669.0
3,Cape Fear,"Nov 15, 1991",NaN,35000000.0,79091969.0,182291969.0
4,Punch-Drunk Love,"Oct 11, 2002",NaN,25000000.0,17791031.0,24591031.0


In [14]:
df_joint_rev['release_date_year'] = pd.DatetimeIndex(df_joint_rev['release_date']).year

In [15]:
df_joint_rev.head()

,movie_title,release_date,year,production_budget,domestic_gross,worldwide_gross,release_date_year
0,How She Move,"Jan 25, 2008",NaN,17000000.0,7070641.0,8607815.0,2008.0
1,Mata Hari,"Dec 26, 1931",NaN,558000.0,900000.0,900000.0,1931.0
2,Stargate,"Oct 28, 1994",NaN,55000000.0,71565669.0,196565669.0,1994.0
3,Cape Fear,"Nov 15, 1991",NaN,35000000.0,79091969.0,182291969.0,1991.0
4,Punch-Drunk Love,"Oct 11, 2002",NaN,25000000.0,17791031.0,24591031.0,2002.0


In [16]:
df_joint_rev = df_joint_rev.drop(['release_date'], axis=1)

In [17]:
#confirm appropriate joins by creating new df that checks year = year
df_joint_rev1 = df_joint_rev.dropna(axis=0,subset = ['year','release_date_year'])
#drop all na so only left with rows with values for both year columns - to confirm year matches
df_joint_rev1.head()

,movie_title,year,production_budget,domestic_gross,worldwide_gross,release_date_year
374,It's a Wonderful Afterlife,2010.0,10000000.0,0.0,1300000.0,2010.0
784,All the Boys Love Mandy Lane,2013.0,750000.0,0.0,1900000.0,2013.0
4540,Jackpot,2014.0,400000.0,800.0,1100800.0,2001.0
4546,Khumba,2013.0,15000000.0,1000.0,27201000.0,2013.0
4548,Skin Trade,2015.0,9000000.0,1200.0,1200.0,2015.0


In [18]:
df_notequal = df_joint_rev1.drop(df_joint_rev1[df_joint_rev1['year'] == df_joint_rev1['release_date_year']].index, inplace = True)
#drop all values that don't have matching year - error in match

/Users/kcahlers/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [19]:
df_joint_rev1
#no years where year != release year
#what should we do with these? 
df_joint_rev1.sort_values(by='worldwide_gross',ascending=False)
#highest gross is karate kid ~360M - this is nothing in comparison to Avatar and titanic (2B)
# fine to drop

,movie_title,year,production_budget,domestic_gross,worldwide_gross,release_date_year
7765,The Karate Kid,2010.0,8000000.0,176600000.0,359100000.0,1984.0
7637,Robin Hood,2010.0,99000000.0,105300000.0,321700000.0,2018.0
7384,Fantastic Four,2015.0,87500000.0,56100000.0,168000000.0,2005.0
7440,Unknown,2011.0,3700000.0,63700000.0,130799999.0,2006.0
7065,The Iron Lady,2011.0,14000000.0,30000000.0,114900000.0,2012.0
6718,The Girl with the Dragon Tattoo,2010.0,90000000.0,10100000.0,104400000.0,2011.0
7339,Selma,2014.0,20000000.0,52100000.0,66800000.0,2015.0
6493,A Monster Calls,2016.0,43000000.0,3700000.0,47300000.0,2017.0
6302,Legend,2015.0,25000000.0,1900000.0,43000000.0,1986.0
7242,Snitch,2013.0,850000.0,42900000.0,42900000.0,2012.0


In [20]:
df_joint_rev=df_joint_rev.drop([7765,
7637,
7384,
7441,
7065,
6718,
7339,
6488,
6302,
7242,
6715,
7128,
6349,
5109,
6785,
5997,
6012,
6611,
6267,
6421,
6474,
6227,
6225,
4538,
4897,
5566,
5390,
5458,
5211,
5161,
4892,
4574])

In [21]:
df_joint_rev

,movie_title,year,production_budget,domestic_gross,worldwide_gross,release_date_year
0,How She Move,NaN,17000000.0,7070641.0,8.607815e+06,2008.0
1,Mata Hari,NaN,558000.0,900000.0,9.000000e+05,1931.0
2,Stargate,NaN,55000000.0,71565669.0,1.965657e+08,1994.0
3,Cape Fear,NaN,35000000.0,79091969.0,1.822920e+08,1991.0
4,Punch-Drunk Love,NaN,25000000.0,17791031.0,2.459103e+07,2002.0
...,...,...,...,...,...,...
7886,Marvel's The Avengers,2012.0,NaN,623400000.0,1.518900e+09,NaN
7887,Jurassic World,2015.0,215000000.0,652300000.0,6.523000e+08,2015.0
7888,Avengers: Infinity War,2018.0,300000000.0,678800000.0,6.788000e+08,2018.0
7889,Black Panther,2018.0,200000000.0,700100000.0,1.347000e+09,2018.0


In [22]:
df_joint_rev = df_joint_rev.assign(release_year=np.where(df_joint_rev.year.isnull(),df_joint_rev.release_date_year , df_joint_rev.year))

In [23]:
df_joint_rev = df_joint_rev.drop(['year'], axis=1)
df_joint_rev = df_joint_rev.drop(['release_date_year'], axis=1)


In [26]:
df_joint_rev

,movie_title,production_budget,domestic_gross,worldwide_gross,release_year
0,How She Move,17000000.0,7070641.0,8.607815e+06,2008.0
1,Mata Hari,558000.0,900000.0,9.000000e+05,1931.0
2,Stargate,55000000.0,71565669.0,1.965657e+08,1994.0
3,Cape Fear,35000000.0,79091969.0,1.822920e+08,1991.0
4,Punch-Drunk Love,25000000.0,17791031.0,2.459103e+07,2002.0
...,...,...,...,...,...
7886,Marvel's The Avengers,NaN,623400000.0,1.518900e+09,2012.0
7887,Jurassic World,215000000.0,652300000.0,6.523000e+08,2015.0
7888,Avengers: Infinity War,300000000.0,678800000.0,6.788000e+08,2018.0
7889,Black Panther,200000000.0,700100000.0,1.347000e+09,2018.0


In [ ]:
df_joint_rev.to_csv('joined_revenue')